In [81]:
import pandas as pd
import re
from sklearn.metrics import accuracy_score, f1_score

In [6]:
'''
a. Load the data into a dataframe. 
    Name it "jungyu_df", and examine the data. 
    You will notice that the file has a header and four tweets with their sentiments.
'''

filepath = 'C:/Users/Public/6th/NLP and Recommender Systems/Assignment1/COVID19_data.csv'

jungyu_df = pd.read_csv(filepath)
jungyu_df.head()

,sentiment,text,user
0,positive,"RT @ashish_vaidya1: @Bhupendrapbjp sir, keshod...",ashish_vaidya1
1,neutral,RT @juank23_7: @Minsa_Peru #DatosAbiertos 🇵🇪 \...,viralvideovlogs
2,neutral,RT @juank23_7: @Minsa_Peru #DatosAbiertos 🇵🇪 \...,Theexpansefan1
3,neutral,RT @RealOzSAGE: Backed by Science: Here’s How ...,sterzie
4,neutral,RT @WHO: The Technical Advisory Group on SARS-...,Gabriel82137261


In [7]:
'''
b. Drop the user column.
'''
jungyu_df = jungyu_df.drop(columns=['user'])

In [12]:
'''
c. Use regular expressions or python string methods to get rid of the additional data at the begging and end of each tweet.
'''
def clean_tweet(text):
    # Remove "RT @username: " at the beginning
    text = re.sub(r'^RT @\w+: ', '', text)
    # Remove all other @username mentions
    text = re.sub(r'@\w+', '', text)
    # Remove trailing ellipsis
    text = re.sub(r'…$', '', text)
    # Remove any characters that are not letters, numbers, or spaces (including standalone numbers)
    text = re.sub(r'[^\w\s]', '', text)
    # Remove standalone numbers
    text = re.sub(r'\b\d+\b', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove any leading or trailing whitespace
    return text.strip()

jungyu_df['text'] = jungyu_df['text'].apply(clean_tweet)

In [13]:
jungyu_df.head()

,sentiment,text
0,positive,sir keshod mamlatdar office says they didnt re...
1,neutral,DatosAbiertos nAl por fecha de defunciónnnEl p...
2,neutral,DatosAbiertos nAl por fecha de defunciónnnEl p...
3,neutral,Backed by Science Heres How We Can Eliminate C...
4,neutral,The Technical Advisory Group on SARSCoV2 Virus...


In [33]:
'''
d. Carry out some basic data exploration and note all the results in your analysis report.
'''
print('Shape of the DataFrame:\n')
print(jungyu_df.shape)

print('\nData Types:\n')
print(jungyu_df.dtypes)

print('\nMissing Values:\n')
print(jungyu_df.isnull().sum())

print('\nTweet Text Describe:\n')
print(jungyu_df['text'].describe())

print('\nSentiment Distribution:\n')
print(jungyu_df['sentiment'].value_counts())

print('\nNumber of Duplicate Rows:')
print(jungyu_df.duplicated().sum())

print('\nTweet Length Statistics:')
print(jungyu_df['text'].apply(len).describe())

Shape of the DataFrame:

(100, 2)

Data Types:

sentiment    object
text         object
dtype: object

Missing Values:

sentiment    0
text         0
dtype: int64

Tweet Text Describe:

count                                                   100
unique                                                   87
top       ACTUALIZACIÓN Esta es la situación de la COVID...
freq                                                      7
Name: text, dtype: object

Sentiment Distribution:

neutral     65
positive    21
negative    14
Name: sentiment, dtype: int64

Number of Duplicate Rows:
13

Tweet Length Statistics:
count    100.000000
mean     104.110000
std       22.614421
min       18.000000
25%      100.750000
50%      109.500000
75%      117.000000
max      136.000000
Name: text, dtype: float64


In [34]:
'''
e. Add a column to your dataframe to reflect the length of each tweet, name it “tweet_len”.
'''
jungyu_df['tween_len'] = jungyu_df['text'].apply(len)
jungyu_df.head()

,sentiment,text,tween_len
0,positive,sir keshod mamlatdar office says they didnt re...,101
1,neutral,DatosAbiertos nAl por fecha de defunciónnnEl p...,95
2,neutral,DatosAbiertos nAl por fecha de defunciónnnEl p...,95
3,neutral,Backed by Science Heres How We Can Eliminate C...,88
4,neutral,The Technical Advisory Group on SARSCoV2 Virus...,123


In [72]:
'''
f. Load the positive and negative words lexicons into two dataframe
'''
positive_words_path = 'C:/Users/Public/6th/NLP and Recommender Systems/Assignment1/positive-words.txt'
negative_words_path = 'C:/Users/Public/6th/NLP and Recommender Systems/Assignment1/negative-words.txt'

positive_df = pd.read_csv(positive_words_path, header=None, names=['word'], encoding='latin1')
negative_df = pd.read_csv(negative_words_path, header=None, names=['word'], encoding='latin1')

print(positive_df.head())
print(negative_df.head())

        word
0         a+
1     abound
2    abounds
3  abundance
4   abundant
         word
0     2-faced
1     2-faces
2    abnormal
3     abolish
4  abominable


In [74]:
'''
g. Iterate through all of the words in each tweet and hit against the list of lexicons in the positive and negative word datframes. 
    Since there are longer tweets, you need to normalize the number of positive and negative hits by the number of words in each tweet.
'''

positive_scores = []
negative_scores = []

positive_words = set(positive_df['word'])
negative_words = set(negative_df['word'])

for tweet in jungyu_df['text']:
    words = tweet.split()
    
    positive_hits = sum(1 for word in words if word in positive_words)
    negative_hits = sum(1 for word in words if word in negative_words)
    
    tweet_len = len(words)
    positive_score = positive_hits / tweet_len if tweet_len > 0 else 0
    negative_score = negative_hits / tweet_len if tweet_len > 0 else 0
    
    positive_scores.append(positive_score)
    negative_scores.append(negative_score)

In [75]:
'''
h. Add two columns to your datafrme one to reflect the percentage of positiveness 
    and the second to reflect the percentage of negativeness.
'''
jungyu_df['positive_score'] = positive_scores
jungyu_df['negative_score'] = negative_scores

In [76]:
'''
i. Add a column to your data frame name it "predicted_sentiment_score".
  i. If both positive and negative percentages are equal to zero, or if both percentages are equal (have the same percentage) tag the tweet as neutral
  ii. If the positive percentage is greater than the negative percentage then tag the tweet as positive.
  iii. If the negative percentage is greater than the positive percentage then tag the tweet as negative.
'''
def tag_sentiment(positive_score, negative_score):
    if positive_score == 0 and negative_score == 0:
        return 'neutral'
    elif positive_score == negative_score:
        return 'neutral'
    elif positive_score > negative_score:
        return 'positive'
    else:
        return 'negative'

jungyu_df['predicted_sentiment_score'] = jungyu_df.apply(lambda row: tag_sentiment(row['positive_score'], row['negative_score']), axis = 1)

In [80]:
# pd.set_option('display.max_rows', None)
# print(jungyu_df)

In [82]:
'''
j. Compare the original sentiments.  
'''

accuracy = accuracy_score(jungyu_df['sentiment'], jungyu_df['predicted_sentiment_score'])
print(accuracy)

0.72


In [84]:
f1 = f1_score(jungyu_df['sentiment'], jungyu_df['predicted_sentiment_score'], average='weighted')
print(f1)

0.6984414017173358
